In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Netherlands'
chdir(csv_file_path)

Y_all = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.', delimiter=',',index_col=0, header=0,parse_dates=True)

Y_temp = Y_all['2000-01-01' : '2015-09-10']
X = X_all['2000-01-01' : '2015-09-10']
print(X.shape, Y_temp.shape)
print(X[~(X.index.isin(Y_temp.index))].shape)
Y = Y_temp.reindex(pd.date_range(start=Y_temp.index.min(),end=Y_temp.index.max(),freq='1D')) 
Y.interpolate(method='linear', inplace=True)  
print(X.shape, Y.shape)
print(Y.isna().sum().sum())

(5732, 9) (5696, 1)
(36, 9)
(5732, 9) (5732, 1)
0


In [2]:
X_train = X[:'2011-12-31']
X_test = X['2014-01-01':]
X_valid = X['2012-01-01':'2013-12-31']

Y_train = Y[:'2011-12-31']
Y_test = Y['2014-01-01':]
Y_valid = Y['2012-01-01':'2013-12-31']

csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *
(dataset_train,dataset_test,dataset_valid,scaler_X,scaler_Y)=prepare_data(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)
 
iters=3
n_steps_in=30
n_steps_out=1

x, y = split_sequences_y1(dataset_train, n_steps_in, n_steps_out)
x_test, y_test = split_sequences_y1(dataset_test, n_steps_in, n_steps_out)
x_valid, y_valid = split_sequences_y1(dataset_valid, n_steps_in, n_steps_out)

In [3]:
import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [4]:
from keras_tuner import HyperModel
from tensorflow.keras import layers

class Transformer_HyperModel(HyperModel):
    def __init__(self, input_shape, output_shape):
        super(Transformer_HyperModel, self).__init__()
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
    def build(self, hp):
        head_size = hp.Int("head_size", min_value=8, max_value=64) #, default=16
        num_heads = 1
        ff_dim = hp.Int("ff_dim", min_value=1, max_value=8) #, default=4
        num_transformer_blocks = 2 #, default=4
        mlp_units = hp.Int("mlp_units", min_value=10, max_value=200)
        dropout = 0.1
        mlp_dropout = 0.1
        
        model = build_model(self.input_shape, self.output_shape, head_size,num_heads,ff_dim,
                                num_transformer_blocks,mlp_units,mlp_dropout)
        model.compile(loss="mean_absolute_error",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),metrics=["mean_absolute_error"])
        return model

input_shape = x.shape[1:]
output_shape = y.shape[1]

hyper_model = Transformer_HyperModel(input_shape, output_shape)

es_callback = keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error',restore_best_weights=True, patience=4)

In [5]:
tuner= kt.RandomSearch(
        hyper_model,
        overwrite=True,
        objective='val_mean_absolute_error',
        max_trials = 10,
        directory='D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge',
        project_name='TF Netherlands'
        )

tuner.search(
        x,
        y,
        batch_size=24,
        epochs=30,
        validation_data=(x_valid,y_valid),
        callbacks=[es_callback],
        verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]

Y_pred=best_model.predict(x_test)
best_model.summary()

Trial 10 Complete [00h 03m 40s]
val_mean_absolute_error: 0.058795493096113205

Best val_mean_absolute_error So Far: 0.05589505657553673
Total elapsed time: 00h 29m 06s
INFO:tensorflow:Oracle triggered exit


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 9)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 9)        18          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 9)        2427        layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 30, 9)        0           multi_head_attention[0][0]   

In [7]:
tuner.results_summary()

csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)
from tensorflow.keras.models import save_model
best_model.save('TF_Netherlands_tune.h5')

NSE=np.array([[None]*n_steps_out])
MAE=np.array([[None]*n_steps_out])
RMSE=np.array([[None]*n_steps_out])
SAPE=np.array([[None]*n_steps_out])

Y_pred_test=best_model.predict(x_test)

y_correct_utf = scaler_Y.inverse_transform(y_test)
yhat_utf = scaler_Y.inverse_transform(Y_pred_test)
NSE[:]= np.array([1 - sum((yhat_utf[:,i]-y_correct_utf[:,i])**2)/sum((y_correct_utf[:,i]-np.mean(y_correct_utf[:,i]))**2)
                  for i in range(n_steps_out)])
MAE[:]=np.array([mean_absolute_error(y_correct_utf[:,i], yhat_utf[:,i]) for i in range(n_steps_out)])
RMSE[:]=np.array([np.sqrt(np.mean((yhat_utf[:,i]-y_correct_utf[:,i])**2)) for i in range(n_steps_out)])
SAPE[:] = np.array([np.abs((yhat_utf[:,i] - y_correct_utf[:,i])/(0.5*(y_correct_utf[:,i]+yhat_utf[:,i]))).mean()
                         for i in range(n_steps_out)]) #mean absolute percent error

print(np.mean(MAE))

Results summary
Results in D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\TF Netherlands
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
head_size: 62
ff_dim: 2
mlp_units: 180
Score: 0.05589505657553673
Trial summary
Hyperparameters:
head_size: 46
ff_dim: 3
mlp_units: 136
Score: 0.058795493096113205
Trial summary
Hyperparameters:
head_size: 60
ff_dim: 8
mlp_units: 167
Score: 0.06204243376851082
Trial summary
Hyperparameters:
head_size: 37
ff_dim: 2
mlp_units: 128
Score: 0.06816931068897247
Trial summary
Hyperparameters:
head_size: 21
ff_dim: 2
mlp_units: 178
Score: 0.06942184269428253
Trial summary
Hyperparameters:
head_size: 14
ff_dim: 6
mlp_units: 78
Score: 0.0710093155503273
Trial summary
Hyperparameters:
head_size: 38
ff_dim: 5
mlp_units: 40
Score: 0.07446075230836868
Trial summary
Hyperparameters:
head_size: 37
ff_dim: 5
mlp_units: 67
Score: 0.08209089189767838
Trial summary
Hyperparameters:
head_size: 49
f